In [3]:
!pip3 install imgbbpy telegraph

Defaulting to user installation because normal site-packages is not writeable
  Using cached telegraph-2.2.0-py3-none-any.whl (10 kB)


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import imgbbpy
from telegraph import Telegraph
import time
client = imgbbpy.SyncClient('4be231765f27bb66bcf899d958d806a6')


telegraph = Telegraph()
telegraph.create_account(short_name='1337')
def create_tlgrph(title, text):
    tit = ' '.join(title.split()[:8])
    response = telegraph.create_page(
        tit,
        html_content=text.replace('\n', '<br>').replace('h1', 'h3').replace(
            'h2', 'h4').replace('span', '').replace('html', '')
    )
    link = response["url"]
    print(link)
    return link

# sorting

In [5]:
with open('result.json', encoding="utf-8") as f:
    data = json.loads(f.read())
df = pd.DataFrame(data['messages'])
df.head()

,id,type,date,date_unixtime,actor,actor_id,action,title,members,text,...,sticker_emoji,width,height,photo,message_id,edited,edited_unixtime,mime_type,poll,forwarded_from
0,-999999995,service,2020-09-02T10:10:08,1599034208,Illia Illia,user1245273083,create_group,Фізика 7-Г,"[Illia Illia, Андрей, Соня Рог]",,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-999999994,service,2020-09-02T10:13:14,1599034394,☦︎︎ 𝔸𝕝𝕖𝕩 ☦︎︎,user1129069882,join_group_by_link,NaN,NaN,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-999999993,service,2020-09-02T10:13:16,1599034396,Nikol,user965573481,join_group_by_link,NaN,NaN,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-999999992,service,2020-09-02T10:13:16,1599034396,MCFdw_TFT,user970230351,join_group_by_link,NaN,NaN,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-999999991,service,2020-09-02T10:13:16,1599034396,Максим С,user974688435,join_group_by_link,NaN,NaN,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Sorting

In [ ]:
# Messages from teacher + real images
# df= df[(df['from'] == 'Андрей') & (df['photo'].str.contains('/'))]
df= df[(df['text']=='') & (df['photo'].str.contains('/'))]
# df = df[df['text']=='']
# Date to readble by PC view
df['date']=[x[:x.index('T')] for x in df['date']]
df['date']=pd.to_datetime(df['date'],format='%Y-%m-%d')
# delete not useful columns
df=df[['id','date','photo','from']]
df['serv']=df['photo'].apply(lambda x: client.upload(file=x).url)
df

In [ ]:
e_g = df[(df['date'] >= '2021-09-01') & (df['date'] < '2022-06-01') ]
s_g = df[(df['date'] >= '2020-09-01') & (df['date'] < '2021-06-01') ]
s_g=s_g[s_g['date'].dt.dayofweek == 4]
e_g=e_g[(e_g['date'].dt.dayofweek == 0) |  (e_g['date'].dt.dayofweek == 2)]
s_g.to_csv('s_g.csv', index=False)
e_g.to_csv('e_g.csv', index=False)

In [ ]:
grp = s_g.groupby('date')
text = ''
data =[]
for name, group in grp:
    name_ =str(name).split()[0]
    date = '/'.join(name_.split('-')[::-1])
    group = group[group['from'] == list(group['from'])[0]]
    print(name_)
    try:
        text = ''.join([f'<img src="{serv}">'  for image,serv in zip(group['photo'],group['serv'])])
        print(text)
        data.append([name_,date,create_tlgrph(name_, text)])
    except Exception as e:
        print('error', e)
        time.sleep(15)
        text = ''.join([f'<img src="{serv}">'  for image,serv in zip(group['photo'],group['serv'])])
        print(text)
        data.append([name_,date,create_tlgrph(name_, text)])
res_s = pd.DataFrame(data, columns=['date','date_','link'])
res_s.to_csv('res_s.csv', index=False)

In [ ]:

grp = e_g.groupby('date')
text = ''
data =[]
for name, group in grp:
    name_ =str(name).split()[0]
    date = '/'.join(name_.split('-')[::-1])
    group = group[group['from'] == list(group['from'])[0]]
    print(name_)
    try:
        text = ''.join([f'<img src="{serv}">'  for image,serv in zip(group['photo'],group['serv'])])
        print(text)
        data.append([name_,date,create_tlgrph(name_, text)])
    except Exception as e:
        print('error', e)
        time.sleep(15)
        text = ''.join([f'<img src="{serv}">'  for image,serv in zip(group['photo'],group['serv'])])
        print(text)
        data.append([name_,date,create_tlgrph(name_, text)])
res_e = pd.DataFrame(data, columns=['date','date_','link'])
res_e.to_csv('res_e.csv', index=False)